In [1]:
# Step 1: Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Path to the dataset on Google Drive (modify this path to where your dataset is located)
dataset_dir = '/content/drive/My Drive/Assess-1-imageDataset'

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Step 2: Load the dataset from the directories
forest_dir = os.path.join(dataset_dir, 'FOREST')
mountain_dir = os.path.join(dataset_dir, 'MOUNTAIN')

# List all images in the directories and assign labels
forest_images = [(os.path.join(forest_dir, img), 0) for img in os.listdir(forest_dir)]
mountain_images = [(os.path.join(mountain_dir, img), 1) for img in os.listdir(mountain_dir)]

# Combine the dataset
all_images = forest_images + mountain_images

# Step 3: Preprocessing function to load, resize, and normalize images
def load_image(img_path, target_size=(224, 224)):
    """
    Load an image from the given path, resize it, and normalize it to the range [0, 1].
    """
    img = load_img(img_path, target_size=target_size)  # Resize the image to 224x224
    img = img_to_array(img)  # Convert the image to a numpy array
    img = img / 255.0  # Normalize the image to have pixel values in the range [0, 1]
    return img

# Load all images with resizing to 224x224
images = [load_image(img_path) for img_path, _ in all_images]
labels = [label for _, label in all_images]

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Step 4: Train-test split (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Step 5: Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Step 6: Define the transfer learning models (MobileNetV2, ResNet50, VGG16)
def create_model(base_model):
    # Freeze the base model layers
    base_model.trainable = False

    # Create a new model using the base model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')  # Sigmoid for binary classification
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Models to be used for transfer learning
models = {
    'MobileNetV2': MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet50': ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'VGG16': VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
}

history_dict = {}

# Step 7: Train the models and store results
for model_name, base_model in models.items():
    print(f"Training {model_name} model...")
    model = create_model(base_model)
    
    # Fit the model on the training data
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=10,
        batch_size=32
    )
    
    # Store the history for plotting and analysis later
    history_dict[model_name] = history

# Step 8: Evaluate and plot results (Accuracy and Loss over epochs)
plt.figure(figsize=(15, 7))

# Plot Accuracy
plt.subplot(1, 2, 1)
for model_name, history in history_dict.items():
    plt.plot(history.history['accuracy'], label=f'{model_name} Training Accuracy')
    plt.plot(history.history['val_accuracy'], label=f'{model_name} Validation Accuracy')
plt.title('Epochs vs Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
for model_name, history in history_dict.items():
    plt.plot(history.history['loss'], label=f'{model_name} Training Loss')
    plt.plot(history.history['val_loss'], label=f'{model_name} Validation Loss')
plt.title('Epochs vs Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Step 9: Performance Analysis for each model (Classification report and Confusion Matrix)
for model_name, base_model in models.items():
    print(f"\nEvaluating {model_name}...")

    model = create_model(base_model)

    # Get predictions and evaluate
    y_pred = model.predict(X_val)
    y_pred = (y_pred > 0.5)  # Convert probabilities to binary labels

    print(f"\nClassification report for {model_name}:\n")
    print(classification_report(y_val, y_pred, target_names=['Forest', 'Mountain']))

    # Confusion Matrix
    cm = confusion_matrix(y_val, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Forest', 'Mountain'], yticklabels=['Forest', 'Mountain'])
    plt.title(f'{model_name} Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

# Step 10: Conclusion
# Compare models based on the evaluation results

# Conclusion
Based on the above steps, we have implemented a transfer learning approach using MobileNetV2, ResNet50, and VGG16 models. We trained these models on the given FOREST and MOUNTAIN datasets, applied data augmentation, and visualized the results with accuracy/loss curves and confusion matrices.